In [ ]:
!pip install keybert
from keybert import KeyBERT

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 

In [ ]:
kw_model = KeyBERT(model = 'all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import math
import pandas as pd
import operator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
top_n = 20
dfi = pd.read_excel('/content/drive/My Drive/geeta_new.xlsx')
dfi.fillna(' ', inplace=True)

Mounted at /content/drive


In [ ]:
final_df = pd.DataFrame()
for chapter in range(1, 19):
  print(chapter)
  df = dfi.loc[(dfi['Chapter'] == chapter)]
  score_list = []
  text_list = []
  string_list = []
  keyword_score_map = {}
  keyword_count_map = {}
  for i in range(math.ceil(df.shape[0]/15)):
    if i==0:
      text = ' '.join(df['English Translation By Swami Sivananda'][0:15])
    else:
      if df.shape[0] >= 15*(i+1) - 3:
        text = ' '.join(df['English Translation By Swami Sivananda'][(15*i-3):(15*(i+1)-3)])
      else:
        text = ' '.join(df['English Translation By Swami Sivananda'][(15*i-3):df.shape[0]])
    list1 = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n = top_n, use_mmr=True, diversity=0.5)
    score_list.append(list1)
    text_list.append(text)
    # print(text)
    # print(list1)
    for jj in range(0, min(len(score_list[0]),top_n)):
      # print('keyword - score ', score_list[0][jj])
      # print('keyword ', score_list[0][jj][0])
      # print('score ', score_list[0][jj][1])
      keyword = score_list[0][jj][0]
      string_list.append(keyword)
      keyword_score_map[keyword] = 0
      keyword_count_map[keyword] = 0
  for ii in range(math.ceil(df.shape[0]/15)):
    for jj in range(math.ceil(df.shape[0]/15)):
        list2 = kw_model.extract_keywords(text_list[jj], candidates=string_list[top_n*ii : top_n*(ii+1)], keyphrase_ngram_range=(1, 1), stop_words='english', top_n = top_n, use_mmr=True, diversity=0.3)
        for kk in range(0, min(len(list2),top_n)):
          # print(len(list2))
          keyword = list2[kk][0]
          keyword_score = list2[kk][1]
          keyword_score_map[keyword]+=keyword_score
          keyword_count_map[keyword]+=1
  keyword_score_map = dict( sorted(keyword_score_map.items(), key=operator.itemgetter(1),reverse=True))
  keyword_df = pd.DataFrame(keyword_score_map.items(), columns=['Keyword', 'Score'])
  keyword_df = keyword_df[:10]
  keyword_df['Chapter'] = chapter
  keyword_df['Author'] = 'Swami Sivananda'
  final_df = pd.concat([final_df, keyword_df], axis=0)

final_df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


,Keyword,Score,Chapter,Author
0,arjuna,5.4652,1,Swami Sivananda
1,dhritarashtra,5.4540,1,Swami Sivananda
2,battle,5.4052,1,Swami Sivananda
3,sons,3.4712,1,Swami Sivananda
4,chariots,2.7432,1,Swami Sivananda
...,...,...,...,...
5,delusion,4.4442,18,Swami Sivananda
6,abandons,2.9700,18,Swami Sivananda
7,speech,2.8470,18,Swami Sivananda
8,sannyasa,1.9170,18,Swami Sivananda


In [ ]:
final_df.to_excel('/content/drive/My Drive/sivananda_score.xlsx')

In [ ]:
final_df = pd.DataFrame()
for chapter in range(1, 19):
  print(chapter)
  df = dfi.loc[(dfi['Chapter'] == chapter)]
  score_list = []
  text_list = []
  string_list = []
  keyword_score_map = {}
  keyword_count_map = {}
  for i in range(math.ceil(df.shape[0]/15)):
    if i==0:
      text = ' '.join(df['English Translation By Swami Adidevananda'][0:15])
    else:
      if df.shape[0] >= 15*(i+1) - 3:
        text = ' '.join(df['English Translation By Swami Adidevananda'][(15*i-3):(15*(i+1)-3)])
      else:
        text = ' '.join(df['English Translation By Swami Adidevananda'][(15*i-3):df.shape[0]])
    list1 = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n = top_n, use_mmr=True, diversity=0.5)
    score_list.append(list1)
    text_list.append(text)
    # print(text)
    # print(list1)
    for jj in range(0, min(len(score_list[0]),top_n)):
      # print('keyword - score ', score_list[0][jj])
      # print('keyword ', score_list[0][jj][0])
      # print('score ', score_list[0][jj][1])
      keyword = score_list[0][jj][0]
      string_list.append(keyword)
      keyword_score_map[keyword] = 0
      keyword_count_map[keyword] = 0
  for ii in range(math.ceil(df.shape[0]/15)):
    for jj in range(math.ceil(df.shape[0]/15)):
        list2 = kw_model.extract_keywords(text_list[jj], candidates=string_list[top_n*ii : top_n*(ii+1)], keyphrase_ngram_range=(1, 1), stop_words='english', top_n = top_n, use_mmr=True, diversity=0.3)
        for kk in range(0, min(len(list2),top_n)):
          # print(len(list2))
          keyword = list2[kk][0]
          keyword_score = list2[kk][1]
          keyword_score_map[keyword]+=keyword_score
          keyword_count_map[keyword]+=1
  keyword_score_map = dict( sorted(keyword_score_map.items(), key=operator.itemgetter(1),reverse=True))
  keyword_df = pd.DataFrame(keyword_score_map.items(), columns=['Keyword', 'Score'])
  keyword_df = keyword_df[:10]
  keyword_df['Chapter'] = chapter
  keyword_df['Author'] = 'Swami Adidevananda'
  final_df = pd.concat([final_df, keyword_df], axis=0)

final_df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


,Keyword,Score,Chapter,Author
0,arjuna,5.7500,1,Swami Adidevananda
1,dhritarashtra,4.5484,1,Swami Adidevananda
2,battle,4.4348,1,Swami Adidevananda
3,chariot,3.2796,1,Swami Adidevananda
4,bhima,2.4224,1,Swami Adidevananda
...,...,...,...,...
5,body,2.6910,18,Swami Adidevananda
6,abandonment,2.4786,18,Swami Adidevananda
7,vital,2.4558,18,Swami Adidevananda
8,sacrifices,2.4474,18,Swami Adidevananda


In [ ]:
final_df = pd.DataFrame()
for chapter in range(1, 19):
  #rint(chapter)
  df = dfi.loc[(dfi['Chapter'] == chapter)]
  score_list = []
  text_list = []
  string_list = []
  keyword_score_map = {}
  keyword_count_map = {}
  for i in range(math.ceil(df.shape[0]/15)):
    if i==0:
      text = ' '.join(df['English Translation By Shri Purohit Swami'][0:15])
    else:
      if df.shape[0] >= 15*(i+1) - 3:
        text = ' '.join(df['English Translation By Shri Purohit Swami'][(15*i-3):(15*(i+1)-3)])
      else:
        text = ' '.join(df['English Translation By Shri Purohit Swami'][(15*i-3):df.shape[0]])
    list1 = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n = top_n, use_mmr=True, diversity=0.5)
    score_list.append(list1)
    text_list.append(text)
    # print(text)
    # print(list1)
    for jj in range(0, min(len(score_list[0]),top_n)):
      # print('keyword - score ', score_list[0][jj])
      # print('keyword ', score_list[0][jj][0])
      # print('score ', score_list[0][jj][1])
      keyword = score_list[0][jj][0]
      string_list.append(keyword)
      keyword_score_map[keyword] = 0
      keyword_count_map[keyword] = 0
  for ii in range(math.ceil(df.shape[0]/15)):
    for jj in range(math.ceil(df.shape[0]/15)):
        list2 = kw_model.extract_keywords(text_list[jj], candidates=string_list[top_n*ii : top_n*(ii+1)], keyphrase_ngram_range=(1, 1), stop_words='english', top_n = top_n, use_mmr=True, diversity=0.3)
        for kk in range(0, min(len(list2),top_n)):
          # print(len(list2))
          keyword = list2[kk][0]
          keyword_score = list2[kk][1]
          keyword_score_map[keyword]+=keyword_score
          keyword_count_map[keyword]+=1
  keyword_score_map = dict( sorted(keyword_score_map.items(), key=operator.itemgetter(1),reverse=True))
  keyword_df = pd.DataFrame(keyword_score_map.items(), columns=['Keyword', 'Score'])
  keyword_df = keyword_df[:10]
  keyword_df['Chapter'] = chapter
  keyword_df['Author'] = 'Shri Purohit Swami'
  final_df = pd.concat([final_df, keyword_df], axis=0)

final_df

,Keyword,Score,Chapter,Author
0,arjuna,5.9180,1,Shri Purohit Swami
1,battle,4.4056,1,Shri Purohit Swami
2,chariot,3.2916,1,Shri Purohit Swami
3,trumpets,2.2496,1,Shri Purohit Swami
4,bheema,1.6596,1,Shri Purohit Swami
...,...,...,...,...
5,philosophy,4.1082,18,Shri Purohit Swami
6,act,3.3600,18,Shri Purohit Swami
7,relinquishment,3.1596,18,Shri Purohit Swami
8,renounce,2.7594,18,Shri Purohit Swami
